In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key
c = Census(census_key, year=2018)

In [35]:
census_codes_csv = pd.read_csv('../Resources/census_columns.csv')
census_codes_csv.dropna(inplace = True)

codes = []
names = []
for i in range(len(census_codes_csv)):
    names.append(census_codes_csv['Name'][i])
    codes.append(census_codes_csv['Code'][i])

In [ ]:
census_data = c.acs5.get(codes, 
                         {'for': 'zip code tabulation area:*'})

In [37]:
census_df = pd.DataFrame(census_data)
census_df.head()

,B01003_001E,B19013_001E,B19301_001E,B02018_007E,B04006_040E,B04006_044E,B02015_001E,B04006_051E,B02015_011E,B04006_022E,B03001_004E,B05003B_001E,B02015_021E,zip code tabulation area
0,17242.0,13092.0,6999.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,145.0,0.0,00601
1,38442.0,16358.0,9277.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,83.0,1070.0,0.0,00602
2,48814.0,16603.0,11307.0,112.0,27.0,0.0,364.0,66.0,0.0,12.0,68.0,1930.0,0.0,00603
3,6437.0,12832.0,5943.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,149.0,0.0,00606
4,27073.0,19309.0,10220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,696.0,0.0,00610


In [39]:
census_rename_dict = {codes[i]: names[i] for i in range(len(codes))} 
census_rename_dict['zip code tabulation area'] = 'zipcode'
census_df.rename(columns = census_rename_dict,inplace = True)
census_df.head()

,population,household_income,per_capita_income,chinese,french,greek,indian,italian,japanese,brazillian,mexican,african_american,thai,zipcode
0,17242.0,13092.0,6999.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,145.0,0.0,00601
1,38442.0,16358.0,9277.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,83.0,1070.0,0.0,00602
2,48814.0,16603.0,11307.0,112.0,27.0,0.0,364.0,66.0,0.0,12.0,68.0,1930.0,0.0,00603
3,6437.0,12832.0,5943.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,149.0,0.0,00606
4,27073.0,19309.0,10220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,696.0,0.0,00610


In [46]:
census_df.agg(['mean','median','std','sem','max'])

,population,household_income,per_capita_income,chinese,french,greek,indian,italian,japanese,brazillian,mexican,african_american,thai,zipcode
mean,9851.278865,-4.415424e+07,-6.479207e+06,147.131310,236.034662,38.442814,530.820562,509.951027,23.363255,12.648309,1091.177506,1246.344626,6.045441,inf
median,2803.500000,5.250000e+04,2.715100e+04,0.000000,57.000000,0.000000,8.000000,67.000000,0.000000,0.000000,39.000000,32.000000,0.000000,49739
std,14614.856872,1.659040e+08,6.555433e+07,883.700683,463.775934,103.847774,2085.602158,1136.741700,213.236795,100.540948,4355.809564,3897.454919,28.551962,NaN
sem,80.306260,9.120977e+05,3.620957e+05,4.855791,2.548373,0.570627,11.460044,6.246211,1.171701,0.552456,23.934465,21.415881,0.156888,NaN
max,122814.000000,2.500010e+05,4.612790e+05,46205.000000,10934.000000,4498.000000,58759.000000,29680.000000,13071.000000,7440.000000,92912.000000,86679.000000,1509.000000,99929
